In [10]:
from openai import OpenAI, AsyncOpenAI
import os
from dotenv import load_dotenv
import httpx
import json



In [11]:
from dotenv import load_dotenv
import os

api_key_router = os.getenv("OPEN_ROUTER")

os.environ['HTTP_PROXY'] = 'socks5://localhost:1080'
os.environ['HTTPS_PROXY'] = 'socks5://localhost:1080'

In [12]:
import os
import json
import time
import sys
from datetime import datetime
import re
import asyncio
import glob
from dotenv import load_dotenv
from openai import OpenAI
from tqdm.auto import tqdm

# Используй ALL_PROXY для SOCKS5
import os
os.environ['HTTP_PROXY'] = 'socks5://localhost:1080'
os.environ['HTTPS_PROXY'] = 'socks5://localhost:1080'
CHANNELS_FILE = "channels_shorts.txt"

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

client_skor = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key_router,

)


In [13]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key_router,

)

# client = OpenAI(
#     base_url="https://openrouter.ai/api/v1",
#     api_key=api_key_router,

# )




In [ ]:
system_prompt = """
You are a database analyst. 
Your task is to analyze the logical data model, data statistics, 
as well as the structure and statistics of SQL queries.
Provide recommendations for modifying the data structure and queries 
to optimize performance.
Respond with a JSON object containing the following fields:
- ddl: a new set of DDL statements to modify the table structure
- migrations: a set of queries for migrating data
- queries: a set of queries with their identifiers that use the new table structure
"""
# Папка с JSON-файлами
datasets_path = "datasets"
json_files = [f for f in os.listdir(datasets_path) if f.endswith(".json")]

llm_list = [
    "google/gemma-3n-e2b-it:free",
    "deepseek/deepseek-r1-0528-qwen3-8b:free",
    "nousresearch/deephermes-3-llama-3-8b-preview:free",
    "mistralai/mistral-7b-instruct:free",
    "moonshotai/kimi-vl-a3b-thinking:free",
    "google/gemma-3-4b-it:free",
    "nousresearch/deephermes-3-llama-3-8b-preview:free",
    "google/gemma-3n-e4b-it:free",
    "qwen/qwen3-4b:free",
    "qwen/qwen3-8b:free",
    "liquid/lfm-3b",
    "meta-llama/llama-3.2-3b-instruct",
    "meta-llama/llama-3.1-8b-instruct",
    "google/gemma-3n-e4b-it",
    "meta-llama/llama-guard-3-8b",
    "moonshotai/kimi-vl-a3b-thinking",
    "nousresearch/hermes-2-pro-llama-3-8b",
    "mistralai/mistral-7b-instruct",
    "mistralai/mistral-7b-instruct-v0.3",
    "meta-llama/llama-3-8b-instruct",
    "deepseek/deepseek-r1-distill-llama-8b",
    "mistralai/ministral-3b",
    "sao10k/l3-lunaris-8b",
    "qwen/qwen3-8b",
    "qwen/qwen-2.5-7b-instruct",
    "cohere/command-r7b-12-2024",
    "google/gemini-flash-1.5-8b",
    "nvidia/nemotron-nano-9b-v2",
    
]

llm_score = {}

def generate_tg_caption(task_input: str, output: str) -> str:
    prompt = f"""You are an evaluator of LLM responses. 
    Evaluate the response strictly on a 10-point scale (1 = worst, 10 = best), based on correctness and completeness.
    
    The LLM receives the following input:
    - DDL statements for creating tables. The script is split into individual queries for each table.
    - A set of queries with statistics on how many times each query was executed.
    - A JDBC connection string with login and password to evaluate the actual data.
    
    The LLM output includes:
    - A new set of DDL queries to modify the table structure.
    - A set of queries for data migration.
    - A set of queries with identifiers that use the new table structure.
    
    Your task: **return only a single integer from 1 to 10** that represents the overall quality of the LLM response. Do not include any explanation, text, or punctuation.
    
    Input: {task_input}
    LLM Response: {output}
    """
    resp = client_skor.chat.completions.create(
        model="x-ai/grok-4-fast",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
        max_tokens=10,
    )
    return resp.choices[0].message.content.strip()

def main():
    for i in range(5):
        print('*'*10, f'Эксперемент № {i+1}','*'*10)
        for llm in llm_list:
            for file_name in json_files:
                file_path = os.path.join(datasets_path, file_name)
                with open(file_path, "r", encoding="utf-8") as f:
                    task_input = json.dumps(json.load(f))
    
                resp = None
                # Попытка сначала с system + user
                try:
                    resp = client.chat.completions.create(
                        model=llm,
                        messages=[
                            {"role": "system", "content": system_prompt},
                            {"role": "user", "content": task_input}
                        ],
                        extra_headers={
                            "HTTP-Referer": "http://localhost",
                            "X-Title": "My Test Script"
                        }
                    )
                except Exception as e:
                    if "Developer instruction is not enabled" in str(e):
                        try:
                            resp = client.chat.completions.create(
                                model=llm,
                                temperature=0.0,
                                messages=[
                                    {"role": "user", "content": f"{system_prompt}\n{task_input}"}
                                ],
                                extra_headers={
                                    "HTTP-Referer": "http://localhost",
                                    "X-Title": "My Test Script"
                                }
                            )
                        except Exception as inner_e:
                            print(f"Ошибка на {llm} с файлом {file_name}: {inner_e}")
                            continue  # пропускаем этот файл для текущего LLM
                    else:
                        print(f"Ошибка на {llm} с файлом {file_name}: {e}")
                        continue  # пропускаем этот файл для текущего LLM
    
                if resp is None:
                    print(f"Пропущено: {llm} {file_name} (нет ответа)")
                    continue
    
                output = resp.choices[0].message.content
                res = generate_tg_caption(task_input, output)
                llm_score.setdefault(i+1, {})  
                llm_score[i+1].setdefault(llm, {})  
                llm_score[i+1][llm][file_name] = res  
                print(f'{llm}: {file_name}: {res}')
            print('_'*80)




main()


********** Эксперемент № 1 **********
google/gemma-3n-e2b-it:free: flights.json: 1
google/gemma-3n-e2b-it:free: questsH.json: 4
________________________________________________________________________________
deepseek/deepseek-r1-0528-qwen3-8b:free: flights.json: 4
deepseek/deepseek-r1-0528-qwen3-8b:free: questsH.json: 3
________________________________________________________________________________
nousresearch/deephermes-3-llama-3-8b-preview:free: flights.json: 2
nousresearch/deephermes-3-llama-3-8b-preview:free: questsH.json: 1
________________________________________________________________________________
mistralai/mistral-7b-instruct:free: flights.json: 9
mistralai/mistral-7b-instruct:free: questsH.json: 1
________________________________________________________________________________
moonshotai/kimi-vl-a3b-thinking:free: flights.json: 3
moonshotai/kimi-vl-a3b-thinking:free: questsH.json: 1
________________________________________________________________________________
google/

### Эксперементы

In [ ]:
"""
google/gemma-3n-e2b-it:free: flights.json: 8
google/gemma-3n-e2b-it:free: questsH.json: 4
________________________________________________________________________________
deepseek/deepseek-r1-0528-qwen3-8b:free: flights.json: 8
deepseek/deepseek-r1-0528-qwen3-8b:free: questsH.json: 8
________________________________________________________________________________
nousresearch/deephermes-3-llama-3-8b-preview:free: flights.json: 7
nousresearch/deephermes-3-llama-3-8b-preview:free: questsH.json: 8
"""

In [13]:
"""
google/gemma-3n-e2b-it:free: flights.json: 5
google/gemma-3n-e2b-it:free: questsH.json: 5
________________________________________________________________________________
deepseek/deepseek-r1-0528-qwen3-8b:free: flights.json: 8
deepseek/deepseek-r1-0528-qwen3-8b:free: questsH.json: 4
________________________________________________________________________________
nousresearch/deephermes-3-llama-3-8b-preview:free: flights.json: 6
nousresearch/deephermes-3-llama-3-8b-preview:free: questsH.json: 9
"""

113.8

In [ ]:
"""
google/gemma-3n-e2b-it:free: flights.json: 6
google/gemma-3n-e2b-it:free: questsH.json: 3
________________________________________________________________________________
deepseek/deepseek-r1-0528-qwen3-8b:free: flights.json: 5
deepseek/deepseek-r1-0528-qwen3-8b:free: questsH.json: 8
________________________________________________________________________________
nousresearch/deephermes-3-llama-3-8b-preview:free: flights.json: 8
Ошибка на nousresearch/deephermes-3-llama-3-8b-preview:free с файлом questsH.json: Expecting value: line 805 column 1 (char 4422)
"""

In [23]:
import pandas as pd

In [ ]:
rows = []
for iteration, llms in llm_score.items():
    for llm, files in llms.items():
        for file_name, score in files.items():
            rows.append({
                "iteration": iteration,
                "llm": llm,
                "file_name": file_name,
                "score": score
            })

df = pd.DataFrame(rows)
df 

In [27]:
df.to_csv('models_results_table_grok.csv', index=False)

,experement,name_model,flights.json,questsH.json
0,1,google/gemma-3n-e2b-it:free,7,3
1,1,deepseek/deepseek-r1-0528-qwen3-8b:free,6,7
2,1,nousresearch/deephermes-3-llama-3-8b-preview:free,9,3
3,1,mistralai/mistral-7b-instruct:free,0,7
4,1,moonshotai/kimi-vl-a3b-thinking:free,3,3
...,...,...,...,...
135,5,qwen/qwen3-8b,0,7
136,5,qwen/qwen-2.5-7b-instruct,6,8
137,5,cohere/command-r7b-12-2024,4,3
138,5,google/gemini-flash-1.5-8b,0,0


In [32]:
df.groupby(['name_model'])[['flights.json','questsH.json']].mean().sort_values(by=["flights.json", "questsH.json"], ascending=[False, False])

,flights.json,questsH.json
name_model,,
google/gemma-3-4b-it:free,8.6,8.0
qwen/qwen3-4b:free,8.6,7.2
nvidia/nemotron-nano-9b-v2,8.2,8.4
liquid/lfm-3b,8.0,7.2
qwen/qwen-2.5-7b-instruct,7.8,7.4
qwen/qwen3-8b:free,7.6,8.0
mistralai/ministral-3b,7.6,6.2
meta-llama/llama-3.1-8b-instruct,7.2,6.6
mistralai/mistral-7b-instruct:free,7.2,6.6


In [34]:
df.groupby(['name_model'])[['flights.json','questsH.json']].median().sort_values(by=["flights.json", "questsH.json"], ascending=[False, False])

,flights.json,questsH.json
name_model,,
google/gemma-3-4b-it:free,9.0,8.0
mistralai/mistral-7b-instruct:free,9.0,7.0
qwen/qwen3-4b:free,9.0,7.0
liquid/lfm-3b,8.0,8.0
nvidia/nemotron-nano-9b-v2,8.0,8.0
qwen/qwen-2.5-7b-instruct,8.0,8.0
qwen/qwen3-8b,8.0,8.0
qwen/qwen3-8b:free,8.0,8.0
deepseek/deepseek-r1-0528-qwen3-8b:free,8.0,7.0


In [ ]:
ПОДВАл

## Подключение к Trino из Python

In [ ]:
import trino

conn = trino.dbapi.connect(
    host='localhost',
    port=8000,
    user='test',
    catalog='flights',
    schema='default'
)
cursor = conn.cursor()
cursor.execute("SHOW CATALOGS")
print(cursor.fetchall())

## Выполнение DDL из JSON

In [ ]:
import re
import json
import trino

# ---------- Параметры соединения ----------
conn = trino.dbapi.connect(
    host='localhost',
    port=8000,
    user='test',
    catalog='flights',    # у тебя должен быть catalog flights
    schema='public'       # default schema when connecting
)
cursor = conn.cursor()

# ---------- Утилиты ----------
_with_re = re.compile(r'\s+WITH\s*\(\s*[^)]*\)\s*;?$', re.IGNORECASE | re.DOTALL)
_create_table_re = re.compile(r'^\s*CREATE\s+TABLE\s+([^\s(]+)', re.IGNORECASE)

def strip_with_clause(statement: str) -> str:
    s = statement.strip()
    if s.endswith(';'):
        s = s[:-1].rstrip()
    return _with_re.sub("", s)

def parse_fq_name(fq: str):
    """Разбивает fully-qualified имя (catalog.schema.table or schema.table or table)"""
    parts = fq.split('.')
    parts = [p.strip().strip('"') for p in parts]
    if len(parts) == 3:
        return parts[0], parts[1], parts[2]
    if len(parts) == 2:
        return None, parts[0], parts[1]  # no catalog specified
    return None, None, parts[0]

def ensure_schema_exists(catalog, schema):
    """Создаёт схему, если её нет. Используем CREATE SCHEMA IF NOT EXISTS, но ловим ошибки."""
    if not catalog or not schema:
        return
    try:
        cursor.execute(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema}")
        print(f"Ensured schema: {catalog}.{schema}")
    except Exception as e:
        print("Failed to CREATE SCHEMA IF NOT EXISTS:", e)
        # Попробуем без IF NOT EXISTS (на старых версиях)
        try:
            cursor.execute(f"CREATE SCHEMA {catalog}.{schema}")
            print(f"Created schema: {catalog}.{schema}")
        except Exception as e2:
            print("CREATE SCHEMA also failed:", e2)
            raise

# ---------- Основной цикл: применяем DDL из JSON ----------
with open("datasets/flights.json", "r", encoding="utf-8") as f:
    task_input = json.load(f)

for ddl_obj in task_input.get("ddl", []):
    orig = ddl_obj["statement"]
    statement = orig.strip().rstrip(';')
    print("\n--- DDL attempt ---")
    print(statement[:200], "..." if len(statement) > 200 else "")
    try:
        cursor.execute(statement)
        print("Executed original DDL successfully.")
        continue
    except Exception as e:
        msg = str(e)
        print("Original DDL failed:", msg)

    # Если ошибка связана с table property (например format) — удаляем WITH(...) и пробуем
    try:
        cleaned = strip_with_clause(statement)
        # перед выполнением cleaned: найдём имя таблицы, и создадим схему, если нужно
        m = _create_table_re.search(cleaned)
        if m:
            fq = m.group(1)
            cat, sch, tbl = parse_fq_name(fq)
            # Если DDL в JSON указывает полное имя flights.public.flights,
            # то cat='flights', sch='public'
            # если cat не указан, используем текущее соединение (conn.catalog)
            if not cat:
                cat = cursor._connection._catalog if hasattr(cursor, '_connection') else None
            if sch:
                ensure_schema_exists(cat, sch)
        # Выполняем cleaned DDL
        cursor.execute(cleaned)
        print("Executed cleaned DDL (WITH removed).")
        continue
    except Exception as e2:
        msg2 = str(e2)
        print("Cleaned DDL also failed:", msg2)

        # Если ошибка говорит, что Schema X not found — попробуем создать схему и повторить
        if "Schema" in msg2 and "not found" in msg2:
            # Попробуем извлечь нужный catalog/schema из первоначальной строки
            m = _create_table_re.search(cleaned)
            if m:
                fq = m.group(1)
                cat, sch, tbl = parse_fq_name(fq)
                if not cat:
                    cat = 'flights'  # если не указан, используем flights, но подставь нужный
                if sch:
                    print(f"Attempting to create missing schema {cat}.{sch} and retry...")
                    try:
                        ensure_schema_exists(cat, sch)
                        # повторная попытка
                        cursor.execute(cleaned)
                        print("Executed cleaned DDL after creating schema.")
                        continue
                    except Exception as e3:
                        print("Retry after schema creation failed:", e3)
        # Иначе — печатаем финальную ошибку
        print("Failed to apply DDL. See above errors for details.")


In [ ]:
ddl["statement"].rstrip(";")

In [ ]:
cursor.execute(ddl_statement)
print("Table created in Trino!")(ddl)

## Проверка созданных таблиц

In [ ]:
cursor.execute("SHOW TABLES;")
print(cursor.fetchall())

In [ ]:
# Заполняем таблицы случайными данными (имитация)
for ddl in task_input.get("ddl", []):
    table_name = ddl["statement"].split()[2]  # простая попытка извлечь имя таблицы
    try:
        # получаем кол-во столбцов
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = [col[1] for col in cursor.fetchall()]
        for _ in range(100):  # 100 строк на таблицу
            values = [random.randint(1, 1000) if col.lower() != 'id' else _ for col in columns]
            placeholders = ",".join(["?"]*len(values))
            cursor.execute(f"INSERT INTO {table_name} ({','.join(columns)}) VALUES ({placeholders})", values)
    except Exception as e:
        print("Insert error:", e)
conn.commit()


In [ ]:
def run_query_real(query, frequency=1, timeout=600):
    """Прогоняем запрос и замеряем время выполнения"""
    try:
        start = time.time()
        cursor.execute(query)
        cursor.fetchall()  # получаем результат
        duration = time.time() - start
        if duration > timeout:
            duration = timeout
    except Exception:
        duration = timeout
    
    # ресурсы (для SQLite синтетически)
    resources_used = random.randint(50, 200)  # MB
    return {
        "queryid": query[:8],
        "frequency": frequency,
        "time_before": duration,
        "time_after": duration,
        "resources_before": resources_used,
        "resources_after": resources_used
    }

# Прогон всех запросов
before_metrics = [run_query_real(q["query"], q.get("runquantity", 1)) for q in task_input.get("queries", [])]
print("Before metrics:", before_metrics)


In [ ]:
import asyncio
from openai import AsyncOpenAI
import nest_asyncio
from dotenv import load_dotenv
import os
import httpx
import json

load_dotenv()
# api_key = os.getenv("OPENAI_API_KEY")
# client = OpenAI(api_key=api_key)
# async_client = AsyncOpenAI(
#     api_key=os.getenv("OPENAI_API_KEY"),
#     http_client=httpx.AsyncClient(proxies="socks5://localhost:1080")
# )



async_client = AsyncOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPEN_ROUTER"),
    http_client=httpx.AsyncClient(proxies="socks5://localhost:1080")
)

        model="google/gemma-3n-e2b-it:free",

system_prompt = """
You are a database analyst. 
Your task is to analyze the logical data model, data statistics, 
as well as the structure and statistics of SQL queries.
Provide recommendations for modifying the data structure and queries 
to optimize performance.
Respond with a JSON object containing the following fields:
- ddl: a new set of DDL statements to modify the table structure
- migrations: a set of queries for migrating data
- queries: a set of queries with their identifiers that use the new table structure
"""

async def analyze_task(task_input):
    response = await async_client.chat.completions.create(
        # model="gpt-4o-mini",
        model="google/gemma-3n-e2b-it:free",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": json.dumps(task_input)},
        ],
        extra_headers={"Authorization": f"Bearer {os.getenv('OPEN_ROUTER')}"}
    )
    try:
        return json.loads(response.choices[0].message.content)
    except:
        return response.choices[0].message.content

# Получаем рекомендации LLM
llm_result = await analyze_task(task_input)
llm_result

In [ ]:
# Применяем новый DDL
for ddl in llm_result.get("ddl", []):
    try:
        cursor.execute(ddl["statement"])
    except Exception as e:
        print("DDL error after LLM:", e)
conn.commit()

# Применяем миграции
for mig in llm_result.get("migrations", []):
    try:
        cursor.execute(mig["statement"])
    except Exception as e:
        print("Migration error:", e)
conn.commit()

# Замер после оптимизации
after_metrics = []
for q in llm_result.get("queries", []):
    metrics = run_query_real(q["query"], frequency=1)
    after_metrics.append(metrics)


In [ ]:
def compute_x(operations):
    numerator = sum(op["frequency"] * op["time_before"] for op in operations)
    denominator = sum(op["frequency"] * op["time_after"] for op in operations)
    return numerator / denominator if denominator else float('inf')

def compute_y(operations):
    S = sum(op["resources_before"] for op in operations)
    C = sum(op["resources_after"] for op in operations)
    return S / C if C else float('inf')

def compute_z(operations):
    x = compute_x(operations)
    y = compute_y(operations)
    return x * y ** (1/3)

# Объединяем метрики для расчёта
for b, a in zip(before_metrics, after_metrics):
    b["time_after"] = a["time_after"]
    b["resources_after"] = a["resources_after"]

z = compute_z(before_metrics)
print("Overall score z:", z)
